<a href="https://colab.research.google.com/github/Ay1932/Machine-Learning/blob/main/Heart_dieases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load the dataset of Heart attack prediction**

In [13]:
import pandas as pd

# Load the dataset
file_path = "/content/drive/MyDrive/heart_attack_prediction_dataset.csv"
df = pd.read_csv(file_path)

# Display first few rows
print(df.head())

# Check column names & data types
print(df.info())

# Check for missing values
print(df.isnull().sum())


  Patient ID  Age     Sex  Cholesterol Blood Pressure  Heart Rate  Diabetes  \
0    BMW7812   67    Male          208         158/88          72         0   
1    CZE1114   21    Male          389         165/93          98         1   
2    BNI9906   21  Female          324         174/99          72         1   
3    JLN3497   84    Male          383        163/100          73         1   
4    GFO8847   66    Male          318          91/88          93         1   

   Family History  Smoking  Obesity  ...  Sedentary Hours Per Day  Income  \
0               0        1        0  ...                 6.615001  261404   
1               1        1        1  ...                 4.963459  285768   
2               0        0        0  ...                 9.463426  235282   
3               1        1        0  ...                 7.648981  125640   
4               1        1        1  ...                 1.514821  160555   

         BMI  Triglycerides  Physical Activity Days Per Week  

**Preprocessing the data**

In [14]:
from sklearn.preprocessing import LabelEncoder

# Drop unnecessary columns
df = df.drop(columns=["Patient ID", "Country", "Continent", "Hemisphere"])

# Split 'Blood Pressure' into 'Systolic BP' and 'Diastolic BP'
df[['Systolic BP', 'Diastolic BP']] = df["Blood Pressure"].str.split("/", expand=True).astype(float)

# Drop the original 'Blood Pressure' column
df = df.drop(columns=["Blood Pressure"])

# Encode categorical columns
categorical_columns = ["Sex", "Diet"]
encoder = LabelEncoder()
for col in categorical_columns:
    df[col] = encoder.fit_transform(df[col])

# Confirm data preprocessing
print(df.head())


   Age  Sex  Cholesterol  Heart Rate  Diabetes  Family History  Smoking  \
0   67    1          208          72         0               0        1   
1   21    1          389          98         1               1        1   
2   21    0          324          72         1               0        0   
3   84    1          383          73         1               1        1   
4   66    1          318          93         1               1        1   

   Obesity  Alcohol Consumption  Exercise Hours Per Week  ...  Stress Level  \
0        0                    0                 4.168189  ...             9   
1        1                    1                 1.813242  ...             1   
2        0                    0                 2.078353  ...             9   
3        0                    1                 9.828130  ...             9   
4        1                    0                 5.804299  ...             6   

   Sedentary Hours Per Day  Income        BMI  Triglycerides  \
0         